In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Dates Module
from datetime import datetime

#Mussing data proportion

In [ ]:
from typing_extensions import dataclass_transform

def missing_dat(data):
  data = pd.DataFrame(data)
  data_frame = pd.DataFrame({
      'Non-Empty Cells': data.notnull().sum(),
      'NaN_Values': data.isnull().sum(),
      '0_Values': (data == '0').sum(),
      '?_Values': (data == '?').sum()
  })
  data_frame['Percentage of Adequate Data'] = (1 - (data_frame['NaN_Values'] / len(data))) * 100
  data_frame['Percentage of Adequate Data'] = (1 - (data_frame['0_Values'] / len(data))) * 100
  data_frame['Percentage of Adequate Data'] = (1 - (data_frame['?_Values'] / len(data))) * 100

  return data_frame

# Dates and time

In [ ]:
def edad(cadena):
    birth = datetime.strptime(cadena, '%d/%m/%Y') # Fecha nacimiento
    prueba = datetime.strptime('09/08/2020', '%d/%m/%Y') #Fecha presentación examen
    return prueba.year - birth.year # Edad el día de presentación del examen

edad('18/04/2003') # Celda de prueba

# Replace by the median

In [ ]:
def null_classifier(dataframe):
  series = dataframe.isnull().sum() * 100 / dataframe.shape[0]
  return series.sort_values(ascending = True)

In [ ]:
def Nan_Replace_by_median(dataframe, print_percentage=False, replace_by_median=True):
  col_with_nuls = dataframe.columns[dataframe.isnull().any()]
  col_with_nuls = col_with_nuls.tolist()

  for col in col_with_nuls:
    col_per = dataframe[col].value_counts(normalize = True)
    col_tuples = list(zip(col_per.index, col_per.values))
    max_tuple = max(col_tuples, key= lambda x:x[1])
  
  if print_percentage == True:
    for i in col_tuples:
      print(col_tuples[i])
  
  if replace_by_median == True:
    for col in col_with_nuls:
      dataframe[col] = dataframe[col].fillna(value = max_tuple[0])
  
  return dataframe

#ML preprocessing

In [ ]:
from sklearn import preprocessing
from category_encoders.ordinal import OrdinalEncoder

## One hot encoding


In [ ]:
def simple_cat_one_hot(dataframe, col_list):
  one_hot = pd.get_dummies(dataframe, columns = col_list)

## Ordinal encoding

In [ ]:
# Mappings are stipulated in the form of a list of directories

mapping = [
    {"col":"name of the colum to transform",
     "mapping":{"str_1":value_1,"str_2":value_2,"str_3":value_3}
     },
     {"col":"name of the column to transform",
      "mapping":{"str_1":value_1,"str_2":value_2,"str_3":value_3}
     }
]

In [ ]:
def Ordinal_Encoder(dataframe, map=None):
  if map == None:
    encoder = OrdinalEncoder()
  else:
    encoder = OrdinalEncoder(mapping = map)
  transform = encoder.fit_transform(dataframe)
  return transform

## Standarization

In [ ]:
def standar_dat(dataframe, mean=True, std=True):
  scaler = preprocessing.StandardScaler(with_mean=mean,with_std=std)
  transform = scaler.fit_transform(dataframe)
  return pd.DataFrame(transform)

# Outlier detection

In [ ]:
# Outliers de una unica columna
def outliers(dataframe, colum_name):
  q1 = dataframe[colum_name].quantile(0.25)
  q3 = dataframe[colum_name].quantile(0.75)
  IQR =q3-q1
  condition = (dataframe[colum_name] < (q1-1.5*IQR)) | (dataframe[colum_name] > (q3+1.5*IQR))
  outliers_data = dataframe[condition].any(axis=1)
  return outliers_data


# Outliers de la clase de una columna con respecto a otra columna
def outliers_per_col(dataframe, col_name_1, col_name_2):
    iqr = lambda x: x.quantile(0.75) - x.quantile(0.25)
    col_1_IQR = dataframe.groupby(col_name_1)[col_name_2].apply(iqr).to_dict()

    col_1_bounds = {}

    for group, IQR in col_1_IQR.items():
        col_1_bounds[group] = (dataframe.loc[dataframe[col_name_1]==group, col_name_2].quantile(0.25)-1.5*IQR,
                             dataframe.loc[dataframe[col_name_1]==group, col_name_2].quantile(0.75)+1.5*IQR)
  
    outliers = dataframe.loc[dataframe.apply(lambda group: 
                (group[col_name_2] < col_1_bounds[group[col_name_1]][0]) or 
                (group[col_name_2] > col_1_bounds[group[col_name_1]][1]), axis=1)]
    
    return outliers